# Community Peer Explorer Demo

This notebook is an demo of the Locus Peer Explorer package. 
It allows you to find and explore the peers of any MSA in 2015 or 2016. 

The different types of peers that you can explore are :
* **Geographic peers**: finds the n closest MSAs. This is usually what is usually done by our potential users as they look for their neighboors. 
* **Input peers**: this features allows you to find peers based on the concentration of any FMs. This will return the n MSAs that have the most similar distribution of those FMs compared to your MSA. You can also choose to identify peers based on outcomes.
* **Peers based on FMs or outcomes automatically selected**: for all these appoaches, we automatically select a subset of FMs or Outcomes and then find the MSAs that are the most similar to you on this subset of features using a distance metric. 
    * *Top FMs*: this feature will return the peers based on the first top n FMs that are the most prevalent in your MSA. You will be able to specify n, the number of FMs you want to base the peer identification on.
    * *Distinguishing FMs*: the peers returned here are based on the FMs that are distinguishing in your MSA. It will include the most present FMs but also the least present. FMs are ranked based on how much the concentration in your MSA differs from other MSAs. You then choose how many of those distinguishing features you want to base your analysis on.
    * *Distinguishing outcomes*: as before, the peers returned here are based on the outcomes that are distinguishing for that MSA.
    * *Minimum coverage peers*: this features selects enough FMs so that a given percentage of the total employment in your MSA is covered. As before, the FMs are selected from the most present to the least present. 
    * *Local/Non-local*: this selects only the local FMs or non-local FMs (waiting for that definition).

In [1]:
%matplotlib inline
from src.call_finder import *

MSA='New York'
input_n_peers = widgets.IntSlider(value=5,min=0,max=20,step=1,description='# of Peers',orientation='horizontal')
input_n_features = widgets.IntSlider(value=5,min=0,max=30,step=1,description='# of Features',orientation='horizontal')
input_year = widgets.RadioButtons(options=[2015, 2016],description='Year', disabled=False)
input_msa = widgets.Dropdown(options=all_msas,value=name2code(MSA),description='MSA',layout=Layout(width="80%"))
input_fms = widgets.SelectMultiple(options=all_fms,description='FM',layout=Layout(width="80%"))
input_outcomes = widgets.SelectMultiple(options=all_outcomes,description='Outcome',layout=Layout(width="80%"))


OSError: Could not find libspatialindex_c library file

# Geographic Peers

In [ ]:
geo = interact_manual(show_peers, df_data=fixed(df_data), 
                     df_county_dist=fixed(df_county_dist), 
                     df_msa_def= fixed(df_msa_def),
                     msa=input_msa, 
                     n_peers=input_n_peers, year=fixed(2015))

# Input FMs peers

In [ ]:
def show_fms_peers(df_data, msa, year, n_peers, fms, outcomes):
    if fms == 'None': 
        fms = [] 
    else: 
        fms = list(fms)
    if outcomes == 'None' :
        outcomes = [] 
    else:
        outcomes = list(outcomes)
    peers, fms = find.get_peers_from_input(df_data, msa, year, n_peers, fms, outcomes)
    for i in fms:
        vis.quadrant_viz(df_data, msa, [msa]+peers, i, save_fig=False, show=True)
    df_peers = pd.DataFrame({'MSA':[str(x) for x in peers]})
    df_peers['Name'] = df_peers['MSA'].apply(code2name)
    return df_peers
    
inputfms = interact_manual(show_fms_peers, df_data=fixed(df_data),
          msa=input_msa, year=fixed(2015), n_peers=input_n_peers,
                     fms=input_fms, outcomes=input_outcomes)

# Distinguishing features peers 

In [ ]:
def show_disting_peers(df_data, msa, year, n_peers, n_feat):
    peers, fms = find.get_distinguishing_features_peers(df_data, msa, year, n_peers, n_feat)
    print(f"Comparison of {msa} and its peers for the 5 most distinguishing FMs")
    for i in range(0,5): 
        vis.quadrant_viz(df_data, msa, [msa]+peers, fms[i], save_fig=False, show=True)
    df_peers = pd.DataFrame({'MSA':[str(x) for x in peers]})
    df_peers['Name'] = df_peers['MSA'].apply(code2name)
    return df_peers

disting = interact_manual(show_disting_peers, df_data=fixed(df_data),
          msa=input_msa, year=input_year, n_peers=input_n_peers,
                     n_feat=input_n_features)



# Top FMs peers

In [ ]:
def show_top_fms_peers(df_data, msa, year, n_peers, n_fms):
    peers, fms = find.get_top_n_fms_peers(df_data, msa, year, n_peers, n_fms)
    print(f"Comparison of {msa} and its peers for the 5 most present FMs")
    for i in range(0,5): 
        vis.quadrant_viz(df_data, msa, [msa]+peers, fms[i], save_fig=False, show=True)
    df_peers = pd.DataFrame({'MSA':[str(x) for x in peers]})
    df_peers['Name'] = df_peers['MSA'].apply(code2name)
    return df_peers
    
top_fms = interact_manual(show_top_fms_peers, df_data=fixed(df_data),
          msa=input_msa, year=input_year, n_peers=input_n_peers,
                     n_fms=input_n_features)

